In [5]:
#!which python for linux and mac
!where python   

c:\Python312\python.exe
C:\msys64\mingw64\bin\python.exe


In [6]:
import cv2
import numpy as np
import os

def check_opencv():
    print(f"✅ OpenCV Version: {cv2.__version__}")

    try:
        dnn_info = cv2.getBuildInformation()
        if "CUDA" in dnn_info:
            print("🔍 Checking OpenCV DNN build...")
            print(dnn_info.split("CUDA")[1].split("\n")[0].strip())  # Show CUDA status
        else:
            print("⚠️ CUDA information not found in OpenCV build.")
    except Exception as e:
        print(f"❌ Error checking OpenCV build: {e}")

def check_cuda():
    try:
        net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        print("✅ CUDA is available for OpenCV DNN!")
    except cv2.error as e:
        print("❌ CUDA is NOT available for OpenCV DNN.")
        print(f"Error: {e}")

def check_files():
    files = ["yolov4.cfg", "yolov4.weights", "cards.names"]
    missing = [f for f in files if not os.path.exists(f)]
    if missing:
        print(f"❌ Missing files: {', '.join(missing)}")
    else:
        print("✅ All YOLO files are present!")

def check_dependencies():
    print("🔍 Checking dependencies...\n")
    
    # Check OpenCV and CUDA
    check_opencv()
    check_cuda()
    
    # Check if YOLO files exist
    check_files()
    
    print("\n🔍 Dependency check complete!")


In [7]:
# Check for dependencies
check_dependencies()

🔍 Checking dependencies...

✅ OpenCV Version: 4.11.0
⚠️ CUDA information not found in OpenCV build.
✅ CUDA is available for OpenCV DNN!
✅ All YOLO files are present!

🔍 Dependency check complete!


In [8]:
# !! READ ME !!
# If you run this program it turns on your webcam

# NOTES
# TODO: 
# Transfer data collected to Godot for processing
# Transfer new model once refined and completed

# LATER
# When or if we switch to using sockets for reading in real time, write to a json and read in Godot via UDP

# Paths for easy access
cfg_path = "yolov4.cfg"
weights_path = "yolov4.weights"
names_path = "cards.names"

# Load YOLO Model
print("Loading YOLO model...")
net = cv2.dnn.readNet(weights_path, cfg_path)

# Check for CUDA availability
print("Checking for CUDA Enabled Devices...")
cuda_available = cv2.cuda.getCudaEnabledDeviceCount() > 0

if cuda_available:
    print("CUDA is available! Using GPU for inference.")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
else:
    print("CUDA is not available. Using CPU for inference.")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()] # Returns a tuple

# Load class labels
with open(names_path, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Initialize video capture
# If its using the wrong camera for you, try changing this
cap = cv2.VideoCapture(0)  # Use 1 for webcam, 0 for camera. We can also use a video path.

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Prepare input image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    results = []  # Store object data

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                results.append({
                    "id": class_id,  # Object class ID
                    "label": classes[class_id],  # Class name
                    "x": center_x,
                    "y": center_y,
                    "confidence": float(confidence)
                })

                # Draw detection
                cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
                cv2.putText(frame, f"{classes[class_id]} ({center_x}, {center_y})",
                            (center_x - 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255, 0, 0), 2)

    # Display the processed frame
    cv2.imshow("YOLO Object Detection", frame)

    # TODO: Send data to Godot (could use networking with sockets or file writing based solution)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # q to exit
        break

cap.release()
cv2.destroyAllWindows()

Loading YOLO model...
Checking for CUDA Enabled Devices...
CUDA is not available. Using CPU for inference.
